In [ ]:
import pandas as pd
import json
import uuid
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# read files through google drive
users = "/content/drive/MyDrive/fetch_data/users.json"
brands = "/content/drive/MyDrive/fetch_data/brands.json"
receipts = "/content/drive/MyDrive/fetch_data/receipts.json"

In [ ]:
# read json files into dataframes
df_users = pd.read_json(users, lines=True)
df_brands = pd.read_json(brands, lines=True)
df_receipts = pd.read_json(receipts, lines=True)

---
# df_users data quality diagnosis



In [ ]:
print(df_users['state'].value_counts())

state
WI    396
NH     20
AL     12
OH      5
IL      3
KY      1
CO      1
SC      1
Name: count, dtype: int64


In [ ]:
print(df_users.info())
print("------------------------------------------")
print(df_users.describe(include="all"))  # summary statistics for numerical and categorical data
print("------------------------------------------")
print(df_users.head())   # first few rows

# non-null count is not consistent across all the columns
#  -> this indicates that some data are missing and we might need some data imputations

# in the statistical summary we see that there are 212 unique ids out of 495
#  -> This indicates possible duplication or an issue with unique identifiers, which could affect data integrity.

# for consistency date columns like createDate and lastLogin should be standarized to maybe a MM-DD-YYYY format
# _id column should be standarized to only the id for easier analysis

# WI state seems to dominated for most of the column
# this could indicates high cardinaity but low variabilities
# an example of drawback on high cardinality is we might potentially need to increase the storage
# an example of drawback on low variability is we might lose the potential for predicting data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           495 non-null    object
 1   active        495 non-null    bool  
 2   createdDate   495 non-null    object
 3   lastLogin     433 non-null    object
 4   role          495 non-null    object
 5   signUpSource  447 non-null    object
 6   state         439 non-null    object
dtypes: bool(1), object(6)
memory usage: 23.8+ KB
None
------------------------------------------
                                         _id active               createdDate  \
count                                    495    495                       495   
unique                                   212      2                       212   
top     {'$oid': '54943462e4b07e684157a532'}   True  {'$date': 1418998882381}   
freq                                      20    494                        20   

              

In [ ]:
# missing values summary
missing_summary = df_users.isnull().sum().sort_values(ascending=False)
print(missing_summary)

# percentage of missing values
missing_percentage = (df_users.isnull().mean() * 100).sort_values(ascending=False)
print(missing_percentage)

# The most problematic column is lastLogin, with the highest count and percentage of missing values 12.53.
# state and signUpSource also have notable amounts of missing data.

lastLogin       62
state           56
signUpSource    48
_id              0
active           0
createdDate      0
role             0
dtype: int64
lastLogin       12.525253
state           11.313131
signUpSource     9.696970
_id              0.000000
active           0.000000
createdDate      0.000000
role             0.000000
dtype: float64


In [ ]:
import pandas as pd

print(df_users.dtypes)

# Check for numeric columns with string values
for col in df_users.select_dtypes(include=['object']).columns:
    # Check if the column contains dictionaries
    if pd.api.types.is_object_dtype(df_users[col]) and any(isinstance(x, dict) for x in df_users[col] if pd.notna(x)):
        print(f"Column '{col}' contains dictionaries. Unique values cannot be directly computed.")
        # Instead of unique(), you might want to print a few sample dictionaries:
        print(f"Sample values in {col}: {df_users[col].dropna().iloc[:5].to_list()}")
    else:
        print(f"Unique values in {col}: {df_users[col].unique()[:5]}")

_id             object
active            bool
createdDate     object
lastLogin       object
role            object
signUpSource    object
state           object
dtype: object
Column '_id' contains dictionaries. Unique values cannot be directly computed.
Sample values in _id: [{'$oid': '5ff1e194b6a9d73a3a9f1052'}, {'$oid': '5ff1e194b6a9d73a3a9f1052'}, {'$oid': '5ff1e194b6a9d73a3a9f1052'}, {'$oid': '5ff1e1eacfcf6c399c274ae6'}, {'$oid': '5ff1e194b6a9d73a3a9f1052'}]
Column 'createdDate' contains dictionaries. Unique values cannot be directly computed.
Sample values in createdDate: [{'$date': 1609687444800}, {'$date': 1609687444800}, {'$date': 1609687444800}, {'$date': 1609687530554}, {'$date': 1609687444800}]
Column 'lastLogin' contains dictionaries. Unique values cannot be directly computed.
Sample values in lastLogin: [{'$date': 1609687537858}, {'$date': 1609687537858}, {'$date': 1609687537858}, {'$date': 1609687530597}, {'$date': 1609687537858}]
Unique values in role: ['consumer' 'fetch


**Note**: I would normlaly check the outliers using IQR (interquartile range) method to ensure outliers, since most of the data in the users table are categoricals checking for outliers is unnessary.


---
# df_brands data quality diagnosis

In [ ]:
print(df_brands.info())   # Data types and non-null counts
print("------------------------------------------")
print(df_brands.describe(include="all"))  # Summary statistics for numerical and categorical data
print("------------------------------------------")
print(df_brands.head())   # First few rows

"""
1. High Missing Data : categoryCode and topBrand have over 50% missing values
 -> depending on the business needs data imputation or drop data might be a good choice
2. Some rows are labeled as "test brand" in the name column
 -> this indicates that the placeholder data might not serve for our analysis
3. _id and cpg can be parsed to standarize data.
4. cpg consist of multiple dictionary (this also violates 1st normal form where column information should be aotmic)
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   _id           1167 non-null   object 
 1   barcode       1167 non-null   int64  
 2   category      1012 non-null   object 
 3   categoryCode  517 non-null    object 
 4   cpg           1167 non-null   object 
 5   name          1167 non-null   object 
 6   topBrand      555 non-null    float64
 7   brandCode     933 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 73.1+ KB
None
------------------------------------------
                                         _id       barcode category  \
count                                   1167  1.167000e+03     1012   
unique                                  1167           NaN       23   
top     {'$oid': '601ac115be37ce2ead437551'}           NaN   Baking   
freq                                       1           NaN      369  

'\n1. High Missing Data : categoryCode and topBrand have over 50% missing values\n -> depending on the business needs data imputation or drop data might be a good choice\n2. Some rows are labeled as "test brand" in the name column\n -> this indicates that the placeholder data might not serve for our analysis\n3. _id and cpg can be parsed to standarize data.\n4. cpg consist of multiple dictionary (this also violates 1st normal form where column information should be aotmic )\n'

In [ ]:
print(df_brands.dtypes)

_id              object
barcode           int64
category         object
categoryCode     object
cpg              object
name             object
topBrand        float64
brandCode        object
dtype: object


In [ ]:
# missing values summary
missing_summary = df_brands.isnull().sum().sort_values(ascending=False)
print(missing_summary)

# percentage of missing values
missing_percentage = (df_brands.isnull().mean() * 100).sort_values(ascending=False)
print(missing_percentage)

"""
1. Highly Missing Columns:
categoryCode and topBrand have over 50% missing values,
brandCode has 20% missing values
category has 13.3% missing values
-> imputation or drop data might be needed depeding on the business needs
"""

categoryCode    650
topBrand        612
brandCode       234
category        155
_id               0
barcode           0
cpg               0
name              0
dtype: int64
categoryCode    55.698372
topBrand        52.442159
brandCode       20.051414
category        13.281919
_id              0.000000
barcode          0.000000
cpg              0.000000
name             0.000000
dtype: float64


---
# df_receipts data quality diagnosis


In [ ]:
print(df_receipts.info())
print("------------------------------------------")
print(df_receipts.describe(include="all"))  # summary statistics for numerical and categorical data
print("------------------------------------------")
print(df_receipts.head())   # first few rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    float64
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptItemList   679 non-null    object 
 12  rewardsReceiptStatus     1119 non-null   object 
 13  totalSpent               684 non-null    float64
 14  userId                  

Data Quality Issues
Missing Values

Several columns have significant proportions of missing values:
bonusPointsEarned and bonusPointsEarnedReason: Only 544 non-null entries (49%).
finishedDate: 568 non-null (51%).
pointsAwardedDate: 537 non-null (48%).
pointsEarned: 609 non-null (54%).
purchaseDate: 671 non-null (60%).
purchasedItemCount: 635 non-null (57%).
totalSpent: 684 non-null (61%).
rewardsReceiptItemList
Inconsistent Date Formatting

Dates are stored as strings in JSON-like objects (e.g., {'$date': 1609687531000}), requiring transformation to proper date-time formats.
Fields affected: createDate, dateScanned, finishedDate, modifyDate, pointsAwardedDate, and purchaseDate.
Data Type Mismatches

Some columns have incorrect data types:
rewardsReceiptItemList is an object (likely a nested JSON structure), requiring further parsing.
purchasedItemCount and totalSpent are float values but contain potential outliers (e.g., purchasedItemCount max = 689, totalSpent max = 4721.95).
Duplicate and Low-Value Records

Potential duplicates in date fields:
Example: createDate and dateScanned have the same top value ({'$date': 1609687531000}) appearing 3 times each.
bonusPointsEarnedReason and rewardsReceiptItemList contain repetitive or generic entries, reducing informational value.
Inconsistent Cardinality

High cardinality in userId (258 unique values over 1,119 entries) may indicate repeated actions by the same users.
Low cardinality in rewardsReceiptStatus (only 5 unique values, with "FINISHED" appearing 518 times) suggests imbalanced categorical data.
Potential Outliers

Columns pointsEarned and totalSpent have extreme values:
pointsEarned max = 10,199.80 (mean = 585.96, std = 1357.16).
totalSpent max = 4,721.95 (mean = 77.80, std = 347.11).
Requires further investigation for validity.
